In [6]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math

### 1. 决策树构建算法
- ID3（基于信息增益）
- C4.5（基于信息增益比）
- CART（gini指数）

#### entropy：$H(x) = -\sum_{i=1}^{n}p_i\log{p_i}$

#### conditional entropy: $H(Y|X)=\sum{P(Xi)}H(Y|X=Xi)$

#### information gain : $g(D, A)=H(D)-H(D|A)$

#### information gain ratio: $g_R(D, A) = \frac{g(D,A)}{H(A)}$

#### gini index:$Gini(D)=\sum_{k=1}^{K}p_k\log{p_k}=1-\sum_{k=1}^{K}p_k^2$

### 2. 在离散值上构建决策树

In [2]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

dataset, label = create_data()

In [3]:
dataset

[['青年', '否', '否', '一般', '否'],
 ['青年', '否', '否', '好', '否'],
 ['青年', '是', '否', '好', '是'],
 ['青年', '是', '是', '一般', '是'],
 ['青年', '否', '否', '一般', '否'],
 ['中年', '否', '否', '一般', '否'],
 ['中年', '否', '否', '好', '否'],
 ['中年', '是', '是', '好', '是'],
 ['中年', '否', '是', '非常好', '是'],
 ['中年', '否', '是', '非常好', '是'],
 ['老年', '否', '是', '非常好', '是'],
 ['老年', '否', '是', '好', '是'],
 ['老年', '是', '否', '好', '是'],
 ['老年', '是', '否', '非常好', '是'],
 ['老年', '否', '否', '一般', '否']]

In [4]:
label

['年龄', '有工作', '有自己的房子', '信贷情况', '类别']

In [7]:
df = pd.DataFrame(dataset, columns=label)
df

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


#### entropy：$H(x) = -\sum_{i=1}^{n}p_i\log{p_i}$

In [10]:
def cal_ent(dataset):
    length = len(dataset)
    values_dict = {}
    for i in range(length):
        label = dataset[i][-1]
        if label not in values_dict:
            values_dict[label] = 1
        else:
            values_dict[label] += 1
    ent = -sum([(i/length)*math.log(i/length, 2) for i in values_dict.values()])
    return ent

cal_ent(np.array(dataset))

0.9709505944546686

#### conditional entropy: $H(Y|X)=\sum{P(Xi)}H(Y|X=Xi)$

In [11]:
def cal_cond_ent(dataset, axis=0):
    length = len(dataset)
    feature_dict = {}
    for i in range(length):
        feature = dataset[i][axis]
        if feature not in feature_dict:
            feature_dict[feature] = []
        feature_dict[feature].append(dataset[i])
    cond_ent = sum([(len(p)/length)*cal_ent(p) for p in feature_dict.values()])
    return cond_ent

cal_cond_ent(np.array(dataset))

0.8879430945988998

#### information gain : $g(D, A)=H(D)-H(D|A)$

In [12]:
def info_gain(ent, cond_ent):
    return ent - cond_ent

info_gain(cal_ent(np.array(dataset)), cal_cond_ent(np.array(dataset)))

0.08300749985576883

#### information gain ratio: $g_R(D, A) = \frac{g(D,A)}{H(A)}$

In [13]:
def cal_a_ent(dataset, axis=0):
    # 计算D关于特征Ag的熵
    length = len(dataset)
    feature_dict = {}
    for i in range(length):
        feature = dataset[i][axis]
        if feature not in feature_dict:
            feature_dict[feature] = 0
        feature_dict[feature] += 1
    a_ent = -sum([(p/length)*math.log(p/length, 2) for p in feature_dict.values()])
    return a_ent

cal_a_ent(dataset)

1.584962500721156

In [14]:
def info_gain_ratio(dataset, axis=0):
    # C4.5
    ent = cal_ent(dataset)
    cond_ent = cal_cond_ent(dataset, axis)
    infogain = info_gain(ent, cond_ent)
    a_ent = cal_a_ent(dataset, axis)
    ratio = infogain / a_ent
    return ratio

info_gain_ratio(dataset)

0.05237190142858302

#### 测试ID3算法的特征选择

In [15]:
# 给予最大信息增益选择特征进行划分
def info_gain_train(dataset):
    count = len(dataset[0]) - 1 # del label
    ent = cal_ent(dataset)
    info_gain_list = []
    for c in range(count):
        cond_ent = cal_cond_ent(dataset, c)
        igain = info_gain(ent, cond_ent)
        info_gain_list.append((c, igain))
        print('特征{}的信息增益是{}'.format(c, igain))
    
    best = max(info_gain_list, key=lambda x: x[-1])
    return '特征{}的信息增益最大，{}'.format(best[0], best[1])

info_gain_train(dataset)

特征0的信息增益是0.08300749985576883
特征1的信息增益是0.32365019815155627
特征2的信息增益是0.4199730940219749
特征3的信息增益是0.36298956253708536


'特征2的信息增益最大，0.4199730940219749'

#### 测试C4.5算法的特征选择

In [16]:
def info_gain_ratio_train(dataset):
    count = len(dataset[0]) - 1
    info_gain_ratio_list = []
    for c in range(count):
        ratio = info_gain_ratio(dataset, c)
        info_gain_ratio_list.append((c, ratio))
        print('特征{}的信息增益率是{}'.format(c, ratio))
    best = max(info_gain_ratio_list, key=lambda x: x[-1])
    return '特征{}的信息增益率最大，{}'.format(best[0], best[1])
        
info_gain_ratio_train(dataset)

特征0的信息增益率是0.05237190142858302
特征1的信息增益率是0.3524465495205019
特征2的信息增益率是0.4325380677663126
特征3的信息增益率是0.23185388128724224


'特征2的信息增益率最大，0.4325380677663126'

In [19]:
class Node():
    def __init__(self, leaf=True, label=None, feature_name=None, feature=None):
        self.leaf = leaf # 是否是叶子结点
        self.label = label # 类标签
        self.feature_name = feature_name # 当前特征名
        self.feature = feature # 当前特征索引
        self.tree = {} # 子树
        self.result = {'label': self.label, 'feature': self.feature, 'tree': self.tree}
        
    def __repr__(self):
        return '{}'.format(self.result)
    
    def add_node(self, val, node):
        self.tree[val] = node
        
    def predict(self, features):
        if self.leaf:
            return self.label
        # 递归查找
        return self.tree[features[self.feature]].predict(features)
    
class DTree():
    def __init__(self, epsilon=0.1, criterion='ID3'):
        self.epsilon = epsilon
        self.criterion = criterion
        self._tree = {}
    
    @staticmethod
    def cal_ent(dataset):
        # 计算D的经验熵
        length = len(dataset)
        values_dict = {}
        for i in range(length):
            label = dataset[i][-1]
            if label not in values_dict:
                values_dict[label] = 1
            else:
                values_dict[label] += 1
        ent = -sum([(i/length)*math.log(i/length, 2) for i in values_dict.values()])
        return ent
    
    def cal_a_ent(dataset, axis=0):
        # 计算D关于特征Ag的熵
        length = len(dataset)
        feature_dict = {}
        for i in range(length):
            feature = dataset[i][axis]
            if feature not in feature_dict:
                feature_dict[feature] = 0
            feature_dict[feature] += 1
        a_ent = -sum([(p/length)*math.log(p/length, 2) for p in feature_dict.values()])
        return a_ent
    
    def cal_cond_ent(self, dataset, axis=0):
        # 计算条件熵
        length = len(dataset)
        feature_dict = {}
        for i in range(length):
            feature = dataset[i][axis]
            if feature not in feature_dict:
                feature_dict[feature] = []
            feature_dict[feature].append(dataset[i])
        cond_ent = sum([(len(p)/length)*cal_ent(p) for p in feature_dict.values()])
        return cond_ent
    
    @staticmethod
    def info_gain(ent, cond_ent):
        # 计算信息增益
        return ent - cond_ent
    
    def info_gain_ratio(self, dataset, axis=0):
        # C4.5
        ent = self.cal_ent(dataset)
        cond_ent = self.cal_cond_ent(dataset, axis)
        info_gain = self.info_gain(ent, cond_ent)
        a_ent = self.cal_a_ent(dataset, axis)
        ratio = info_gain / a_ent
        return ratio
    
    def info_gain_train(self, dataset):
        # ID3
        count = len(dataset[0]) - 1 # del label
        ent = self.cal_ent(dataset)
        info_gain_list = []
        for c in range(count):
            cond_ent = self.cal_cond_ent(dataset, c)
            igain = self.info_gain(ent, cond_ent)
            info_gain_list.append((c, igain))
            # print('特征{}的信息增益是{}'.format(c, igain))
        
        # best: (索引，信息增益)
        best = max(info_gain_list, key=lambda x: x[-1])
        return best
    
    def info_gain_ratio_train(self, dataset):
        # C4.5
        count = len(dataset[0]) - 1
        info_gain_ratio_list = []
        for c in range(count):
            ratio = info_gain_ratio(dataset, c)
            info_gain_ratio_list.append((c, ratio))
            # print('特征{}的信息增益率是{}'.format(c, ratio))
        best = max(info_gain_ratio_list, key=lambda x: x[-1])
        return best
    
    def train(self, train_data):
        """
        train_data -- dataframe
        """
        
        x_train, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
        # 1. 若训练集数据属于同一个类别Ck，则将Ck作为类标记，建立单节点树T，返回T
        if len(y_train.value_counts()) == 1:
            return Node(leaf=True, label=y_train.iloc[0])
        
        # 2. 若特征集为空，则选择训练集中占比最大的类标签Ck作为节点的标记，建立单节点树T，返回T
        if len(features) == 0:
            return Node(leaf=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # 3. 计算信息收益，选择信息受益最大的特征作为Ag
        if self.criterion == 'ID3':
            max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
            max_feature_name = features[max_feature]
        if self.criterion == 'C4.5':
            max_feature, max_info_gain = self.info_gain_ratio_train(np.array(train_data))
            max_feature_name = features[max_feature]
        
        # 4. 如果Ag的信息增益小于epsilon，则停止决策，将训练集中类别数最大的类标签Ck作为标记，返回T
        if max_info_gain < self.epsilon:
            return Node(leaf=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # 5. 以Ag进行划分，分别有特征类别棵子树
        node_tree = Node(leaf=False, feature_name=max_feature_name, feature=max_feature)
        feature_list = train_data[max_feature_name].value_counts().index.tolist()
        
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)
            
            # 6. 递归生成子树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)
        
        return node_tree
    
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self, X_test):
        """预测单个样本的标签，X_test是单个样本"""
        try:
            return self._tree.predict(X_test) 
        except KeyError:
            return None
    
    def score(self, X, y):
        """计算预测正确率,
        X -- array, shape=(m, n)
        y -- array, shape=(m,)
        """
        
        correct = 0
        for i in range(X.shape[0]):
            x_use = X[i, :]
            y_use = y[i]
            y_pred = self.predict(x_use)
            if y_pred is None:
                continue
            if y_pred == y_use:
                correct += 1
        return correct / X.shape[0]

In [20]:
from sklearn.model_selection import train_test_split

datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)
X_train, X_val, y_train, y_val = train_test_split(data_df.iloc[:, :-1], data_df.iloc[:, -1], test_size=0.3, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
for c in ['ID3', 'C4.5']:
    dt = DTree(criterion=c)
    #tree = dt.fit(data_df)
    tree = dt.fit(pd.concat([X_train, y_train], axis=1))
    print('with criterion(%s) the accuracy in train set is %f' %(c, dt.score(X_train.values, y_train.values)))
    print('with criterion(%s) the accuracy in test set is %f' %(c, dt.score(X_val.values, y_val.values)))

(10, 4) (5, 4) (10,) (5,)
with criterion(ID3) the accuracy in train set is 1.000000
with criterion(ID3) the accuracy in test set is 0.600000
with criterion(C4.5) the accuracy in train set is 0.800000
with criterion(C4.5) the accuracy in test set is 0.600000


In [21]:
def create_data1():
    data=[
            ['young',   'myope',   'no',  'reduced',  'no lenses'],
            ['young',   'myope',    'no',  'normal',  'soft'],
            ['young',   'myope',   'yes', 'reduced', 'no lenses'],
            ['young',   'myope',   'yes',  'normal',  'hard'],
            ['young',   'hyper',   'no',  'reduced', 'no lenses'],
            ['young',   'hyper',   'no',  'normal',  'soft'],
            ['young',   'hyper',   'yes', 'reduced', 'no lenses'],
            ['young',   'hyper',   'yes', 'normal',  'hard'],
            ['pre',        'myope',   'no',  'reduced', 'no lenses'],
            ['pre', 'myope',   'no',  'normal',  'soft'],
            ['pre', 'myope',   'yes', 'reduced',  'no lenses'],
            ['pre', 'myope',   'yes', 'normal',  'hard'],
            ['pre', 'hyper',   'no',  'reduced', 'no lenses'],
            ['pre', 'hyper',   'no',  'normal',  'soft'],
            ['pre', 'hyper',   'yes', 'reduced', 'no lenses'],
            ['pre', 'hyper',   'yes', 'normal',  'no lenses'],
            ['presbyopic',  'myope',   'no',  'reduced', 'no lenses'],
            ['presbyopic',  'myope',   'no',  'normal',  'no lenses'],
            ['presbyopic',  'myope',   'yes', 'reduced', 'no lenses'],
            ['presbyopic',  'myope',   'yes', 'normal',  'hard'],
            ['presbyopic',  'hyper',   'no',  'reduced', 'no lenses'],
            ['presbyopic',  'hyper',   'no',  'normal',  'soft'],
            ['presbyopic',  'hyper',   'yes', 'reduced', 'no lenses'],
            ['presbyopic',  'hyper',   'yes', 'normal',  'no lenses']
            ]

    labels = ['age', 'prescript', 'stigmatic', 'tearRate', 'target']
    return data, labels

In [22]:
from sklearn.model_selection import train_test_split

datasets, labels = create_data1()
data_df = pd.DataFrame(datasets, columns=labels)

X_train, X_val, y_train, y_val = train_test_split(data_df.iloc[:, :-1], data_df.iloc[:, -1], test_size=0.4, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
for c in ['ID3', 'C4.5']:
    dt = DTree(criterion=c)
    tree = dt.fit(pd.concat([X_train, y_train], axis=1))
    #print(tree)
    print('with criterion(%s) the accuracy in train set is %f' %(c, dt.score(X_train.values, y_train.values)))
    print('with criterion(%s) the accuracy in test set is %f' %(c, dt.score(X_val.values, y_val.values)))

(14, 4) (10, 4) (14,) (10,)
with criterion(ID3) the accuracy in train set is 0.857143
with criterion(ID3) the accuracy in test set is 0.800000
with criterion(C4.5) the accuracy in train set is 1.000000
with criterion(C4.5) the accuracy in test set is 0.700000


In [ ]:
'''
dt = DTree()
tree = dt.fit(data_df)
print(dt.score(data_df.values[:, :-1], data_df.values[:, -1]))
for i in range(data_df.shape[0]):
    print(dt.predict(data_df.iloc[i].values))
'''

### 3. iris分类

In [23]:
from sklearn import datasets

data = datasets.load_iris()
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [24]:
x = data['data']
y = data['target']
columns = data['feature_names']
labels = data['target_names']

In [25]:
print(x.shape, y.shape, columns, labels)

(150, 4) (150,) ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'] ['setosa' 'versicolor' 'virginica']


In [29]:
# 说明类别平衡
np.bincount(y)

array([50, 50, 50])

### 3.1 sklearn decisiontree

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
print('sklearn decision tree trainset accuracy: ', dtc.score(x_train, y_train))
print('sklearn decision tree testset accuracy: ', dtc.score(x_test, y_test))

sklearn decision tree trainset accuracy:  1.0
sklearn decision tree testset accuracy:  1.0


In [69]:
data_df = pd.DataFrame(np.hstack((x, np.expand_dims(y, axis=-1))), columns=columns+['traget'])
X_train, X_val, y_train, y_val = train_test_split(data_df.iloc[:, :-1], data_df.iloc[:, -1], test_size=0.3, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
train_score, test_score = [], []
features = []
for c in ['ID3', 'C4.5']:
    for e in [5, 1, 0.6, 0.3, 0.1, 0.01]:
        features.append((c, e))
for c, e in features:
    dt = DTree(criterion=c, epsilon=e)
    tree = dt.fit(pd.concat([X_train, y_train], axis=1))
    #print('with criterion(%s) and epsilon(%f) the accuracy in train set is %f' %(c, e, dt.score(X_train.values, y_train.values)))
    train_score.append(dt.score(X_train.values, y_train.values))
    test_score.append(dt.score(X_val.values, y_val.values))
    #print('with criterion(%s) and epsilon(%f) the accuracy in test set is %f' %(c, e, dt.score(X_val.values, y_val.values)))
print(train_score)
print(test_score)

(105, 4) (45, 4) (105,) (45,)
[0.37142857142857144, 0.9714285714285714, 1.0, 1.0, 1.0, 1.0, 0.37142857142857144, 0.37142857142857144, 0.37142857142857144, 0.9904761904761905, 1.0, 1.0]
[0.24444444444444444, 0.8, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.24444444444444444, 0.24444444444444444, 0.24444444444444444, 0.8888888888888888, 0.8444444444444444, 0.8444444444444444]


In [72]:
trains = np.array(train_score)
tests = np.array(test_score)
# np.hstack((trains, tests)).reshape(4, 5).T
result = pd.DataFrame(np.hstack((trains, tests)).reshape(4, 6).T, columns=['train_ID3', 'train_C4.5', 'test_ID3', 'test_C4.5'], index=[5, 1, 0.6, 0.3, 0.1, 0.01])

In [73]:
result

,train_ID3,train_C4.5,test_ID3,test_C4.5
5.00,0.371429,0.371429,0.244444,0.244444
1.00,0.971429,0.371429,0.800000,0.244444
0.60,1.000000,0.371429,0.733333,0.244444
0.30,1.000000,0.990476,0.733333,0.888889
0.10,1.000000,1.000000,0.733333,0.844444
0.01,1.000000,1.000000,0.733333,0.844444


In [74]:
result.reindex(columns=['train_ID3', 'test_ID3', 'train_C4.5', 'test_C4.5'])

,train_ID3,test_ID3,train_C4.5,test_C4.5
5.00,0.371429,0.244444,0.371429,0.244444
1.00,0.971429,0.800000,0.371429,0.244444
0.60,1.000000,0.733333,0.371429,0.244444
0.30,1.000000,0.733333,0.990476,0.888889
0.10,1.000000,0.733333,1.000000,0.844444
0.01,1.000000,0.733333,1.000000,0.844444
